In [1]:
import time, json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup, SoupStrainer
from bs4.diagnose import diagnose
from parsel import Selector
from selenium.webdriver.common.action_chains import ActionChains

stat_base_url = "https://www.tandfonline.com"

driver = webdriver.Chrome()
action=ActionChains(driver)
CRinBiotech_url = driver.get("https://www.tandfonline.com/loi/ibty20")
CRinBiotech = driver.page_source
time.sleep(5)
list_of_volumes = SoupStrainer(class_='volume_link')
browse_soup = BeautifulSoup(CRinBiotech, 'html.parser', parse_only = list_of_volumes)


#Open each tab and get issue link
volume_links = []

time.sleep(5)
    
for link in browse_soup.find_all('a'):
    volume_links.append(link.get('href'))

issue_links = []    
for volume in volume_links:
    time.sleep(3)
    issue_url = driver.get(stat_base_url + volume)
    issue_ = driver.page_source
    
    list_of_issues = SoupStrainer(class_='issue-link')
    browse_vol_issue_soup = BeautifulSoup(issue_, 'html.parser', parse_only = list_of_issues)

    for issue in browse_vol_issue_soup.find_all('a'):
        issue_links.append(issue.get('href'))

titles = []
doi = []
for i in issue_links:
    article_url = driver.get(stat_base_url + i)
    time.sleep(1)
    article_ = driver.page_source
    list_articles = SoupStrainer(class_='ref nowrap')
    browse_article = BeautifulSoup(article_, 'html.parser', parse_only = list_articles)
    
    for art_title in browse_article.find_all('a'):
        titles.append(art_title.get_text())

    for art_link in browse_article.find_all('a'):
        doi.append(stat_base_url + art_link.get('href'))

article_dict = dict(zip(titles, doi))
with open('CRinBiotech.json', 'w+') as fp:
    json.dump(article_dict, fp,indent = "\n")